# **Implémentation du modèle**
---

## Introduction

Ce notebook contient les codes nécessaire à la mise en place du modèle. Le modèle est inspiré de celui de Iacoviello et Navarro : 

- La première étape consiste à identifier les chocs de politique monétaire, en prenant le résidu de la régression du taux d'intérêt sur l'écart d'inflation par rapport à la cible et l'output gap ou le chômage. En réalité Iacoviello et Navarro utilisent des lags et valeurs présentes de l'inflation, des spreads de crédit, du PIB et des lags des taux des fonds fédéraux.
Il nous manque donc l'output gap, l'inflation et le chômage de la zone euro
- La deuxième étape consiste à estimer l'impact de ces chocs sur l'activité économique (PIB, emploi ...) en régressant ces variables sur les chocs et des variables de contrôles (4 lags du PIB et des trends linéaires et quadratiques)



## Identification des chocs

In [2]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import sklearn.metrics
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

Problème: dans Iacoviello et Navarro, ils régressent le taux d'intérêt sur le PIB etc des US seulement. Or on est dans la zone euro ici, doit-on faire une régression de panel ? 

Pour faire la première étape il nous faudrait un data frame avec les données de seulement la zone euro.
Il faut construire les variables suivantes : 
- log_euroarea_gdp, 
- corporate spreads, 
- lagged_federal_funds (<span style="color:red"> Iacoviello et Navarro utilisent le shadow rate, je pense qu'il faut faire une variable par lag, ils prennent 4 lags dans le papier</span>),
- log_foreign_gdp (<span style="color:red"> le problème c'est qu'on n'a pas de pays étrangers dans notre base donc on ne peut pas vraiment créer cette variable</span> )


Pour l'instant pour le federal funds rate j'ai pris le key interest rate qui est sur le site de l'OCDE

In [3]:
%pip install -r requirements.txt
import pandas as pd
import numpy as np


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Import et formatage du key interest rate
df_key = pd.read_csv("Données_extraites/key interest rates.csv", encoding='utf-8')
df_key = df_key[['TIME_PERIOD', 'OBS_VALUE']]
df_key = df_key.rename(columns={'OBS_VALUE':'key_rate'})
df_key["TIME_PERIOD"] = pd.to_datetime(df_key["TIME_PERIOD"])
df_key = df_key.sort_values(by="TIME_PERIOD")

# création des lags 
df_key["key_rate_lag1"] = df_key["key_rate"].shift(1)
df_key["key_rate_lag2"] = df_key["key_rate"].shift(2)
df_key["key_rate_lag3"] = df_key["key_rate"].shift(3)
df_key["key_rate_lag4"] = df_key["key_rate"].shift(4)
df_key

Index(['STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME', 'ACTION', 'REF_AREA',
       'Zone de référence', 'MEASURE', 'Mesure', 'FREQ',
       'Fréquence d'observation', 'TIME_PERIOD', 'Période de temps',
       'OBS_VALUE', 'Observation value', 'OBS_STATUS', 'Observation status',
       'UNIT_MEASURE', 'Unité de mesure', 'PRICE_BASE', 'Ajustement',
       'ADJUSTMENT', 'Base de prix', 'UNIT_MULT', 'Multiplicateur d'unité',
       'CURRENCY', 'Monnaie', 'BASE_PER', 'Période de base', 'METHODOLOGY',
       'Méthode', 'DECIMALS', 'Decimals'],
      dtype='object')


/tmp/ipykernel_5409/3588946038.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_key["TIME_PERIOD"] = pd.to_datetime(df_key["TIME_PERIOD"])


,TIME_PERIOD,key_rate,key_rate_lag1,key_rate_lag2,key_rate_lag3,key_rate_lag4
6,1999-01-01,2.00,NaN,NaN,NaN,NaN
5,1999-04-01,1.50,2.00,NaN,NaN,NaN
28,1999-07-01,1.50,1.50,2.0,NaN,NaN
27,1999-10-01,2.00,1.50,1.5,2.0,NaN
26,2000-01-01,2.50,2.00,1.5,1.5,2.0
...,...,...,...,...,...,...
39,2023-10-01,4.00,4.00,3.5,3.0,2.0
38,2024-01-01,4.00,4.00,4.0,3.5,3.0
37,2024-04-01,4.00,4.00,4.0,4.0,3.5
25,2024-07-01,3.75,4.00,4.0,4.0,4.0


In [6]:
# import et formatage du GDP

df_euro_gdp = pd.read_csv("Données_extraites/estat_namq_10_gdp_filtered_en (1).csv.gz", compression="gzip")
df_euro_gdp=df_euro_gdp[['TIME_PERIOD', 'OBS_VALUE']]
df_euro_gdp=df_euro_gdp.rename(columns={'OBS_VALUE':'gdp'})
df_euro_gdp["log_gdp"] = np.log(df_euro_gdp["gdp"])

# mettre les dates dans l'ordre
df_euro_gdp["TIME_PERIOD"] = pd.to_datetime(df_euro_gdp["TIME_PERIOD"])
df_euro_gdp = df_euro_gdp.sort_values(by="TIME_PERIOD")

# création des lags pour gdp et log_gdp

df_euro_gdp["gdp_lag1"] = df_euro_gdp["gdp"].shift(1)
df_euro_gdp["gdp_lag2"] = df_euro_gdp["gdp"].shift(2)
df_euro_gdp["gdp_lag3"] = df_euro_gdp["gdp"].shift(3)
df_euro_gdp["gdp_lag4"] = df_euro_gdp["gdp"].shift(4)

df_euro_gdp["log_gdp_lag1"] = df_euro_gdp["log_gdp"].shift(1)
df_euro_gdp["log_gdp_lag2"] = df_euro_gdp["log_gdp"].shift(2)
df_euro_gdp["log_gdp_lag3"] = df_euro_gdp["log_gdp"].shift(3)
df_euro_gdp["log_gdp_lag4"] = df_euro_gdp["log_gdp"].shift(4)
df_euro_gdp



/tmp/ipykernel_5409/1897129056.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_euro_gdp["TIME_PERIOD"] = pd.to_datetime(df_euro_gdp["TIME_PERIOD"])


,TIME_PERIOD,gdp,log_gdp,gdp_lag1,gdp_lag2,gdp_lag3,gdp_lag4,log_gdp_lag1,log_gdp_lag2,log_gdp_lag3,log_gdp_lag4
0,1995-01-01,1340877.4,14.108835,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1995-04-01,1385087.6,14.141274,1340877.4,NaN,NaN,NaN,14.108835,NaN,NaN,NaN
2,1995-07-01,1388312.6,14.143600,1385087.6,1340877.4,NaN,NaN,14.141274,14.108835,NaN,NaN
3,1995-10-01,1472070.7,14.202181,1388312.6,1385087.6,1340877.4,NaN,14.143600,14.141274,14.108835,NaN
4,1996-01-01,1400071.7,14.152034,1472070.7,1388312.6,1385087.6,1340877.4,14.202181,14.143600,14.141274,14.108835
...,...,...,...,...,...,...,...,...,...,...,...
115,2023-10-01,3804526.0,15.151702,3639290.0,3626149.8,3528215.7,3602551.8,15.107299,15.103682,15.076303,15.097153
116,2024-01-01,3662523.2,15.113663,3804526.0,3639290.0,3626149.8,3528215.7,15.151702,15.107299,15.103682,15.076303
117,2024-04-01,3759149.1,15.139703,3662523.2,3804526.0,3639290.0,3626149.8,15.113663,15.151702,15.107299,15.103682
118,2024-07-01,3781949.7,15.145750,3759149.1,3662523.2,3804526.0,3639290.0,15.139703,15.113663,15.151702,15.107299


In [ ]:
# Import et formatage inflation
# pour l'inflation j'ai pris dans l'OCDE, economic outlook 116 -> harmonized core infation, Euro Area(17 countries)

df_euro_inflation = pd.read_csv("Données_extraites/inflation euro.csv", encoding='utf-8')
df_euro_inflation = df_euro_inflation[['TIME_PERIOD', 'OBS_VALUE']]
df_euro_inflation = df_euro_inflation.rename(columns={'OBS_VALUE':'inflation'})
df_euro_inflation["TIME_PERIOD"] = pd.to_datetime(df_euro_inflation["TIME_PERIOD"])
df_euro_inflation = df_euro_inflation.sort_values(by="TIME_PERIOD")

# création des lags 
df_euro_inflation["inflation_lag1"] = df_euro_inflation["inflation"].shift(1)
df_euro_inflation["inflation_lag2"] = df_euro_inflation["inflation"].shift(2)
df_euro_inflation["inflation_lag3"] = df_euro_inflation["inflation"].shift(3)
df_euro_inflation["inflation_lag4"] = df_euro_inflation["inflation"].shift(4)
df_euro_inflation



Index(['STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME', 'ACTION', 'REF_AREA',
       'Zone de référence', 'MEASURE', 'Mesure', 'FREQ',
       'Fréquence d'observation', 'TIME_PERIOD', 'Période de temps',
       'OBS_VALUE', 'Observation value', 'OBS_STATUS', 'Observation status',
       'UNIT_MEASURE', 'Unité de mesure', 'UNIT_MULT',
       'Multiplicateur d'unité', 'CURRENCY', 'Monnaie', 'BASE_PER',
       'Période de base', 'METHODOLOGY', 'Méthode', 'DECIMALS', 'Decimals',
       'PRICE_BASE', 'Ajustement', 'ADJUSTMENT', 'Base de prix'],
      dtype='object')


/tmp/ipykernel_5409/1221002929.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_euro_inflation["TIME_PERIOD"] = pd.to_datetime(df_euro_inflation["TIME_PERIOD"])


,TIME_PERIOD,inflation,inflation_lag1,inflation_lag2,inflation_lag3,inflation_lag4
7,1999-01-01,1.404333,NaN,NaN,NaN,NaN
6,1999-04-01,1.186841,1.404333,NaN,NaN,NaN
5,1999-07-01,1.125569,1.186841,1.404333,NaN,NaN
4,1999-10-01,1.030436,1.125569,1.186841,1.404333,NaN
26,2000-01-01,1.055140,1.030436,1.125569,1.186841,1.404333
...,...,...,...,...,...,...
40,2023-10-01,3.715027,5.042847,5.459302,5.504622,5.094049
39,2024-01-01,3.088601,3.715027,5.042847,5.459302,5.504622
25,2024-04-01,2.784795,3.088601,3.715027,5.042847,5.459302
24,2024-07-01,2.720590,2.784795,3.088601,3.715027,5.042847


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

data = pd.read_pickle('data_final.pkl')


# Sélection des colonnes utiles
colonnes_utiles = ['federal_funds_rate', 'inflation', 'log_euroarea_gdp', 'corporate_spreads', 'log_foreign_gdp', 'lagged_federal_funds', 'time']
data = data[colonnes_utiles]

# Création d'une variable de tendance quadratique
data['time_squared'] = data['time'] ** 2

# Définition de rt=federal funds rate
Y = data['federal_funds_rate']

# Définir les variables indépendantes
X = data[['inflation', 'log_euroarea_gdp', 'corporate_spreads', 'log_foreign_gdp', 'lagged_federal_funds', 'time', 'time_squared']]
X = sm.add_constant(X)  # Ajout d'une constante pour l'intercept

# Regression
model = sm.OLS(Y, X).fit()

# Extraire les résidus comme chocs monétaires
data['monetary_shocks'] = model.resid

# Afficher le résumé de la régression
print(model.summary())

# Sauvegarder les résultats dans un fichier CSV
data[['time', 'monetary_shocks']].to_csv('identified_monetary_shocks.csv', index=False)


KeyError: "None of [Index(['federal_funds_rate', 'inflation', 'log_euroarea_gdp',\n       'corporate_spreads', 'log_foreign_gdp', 'lagged_federal_funds', 'time'],\n      dtype='object')] are in the [columns]"

In [4]:
data

,CPI_Austria,PIB_Austria,LT_IR_Austria,ST_IR_Austria,WH_Austria,CPI_Belgium,PIB_Belgium,LT_IR_Belgium,ST_IR_Belgium,WH_Belgium,...,CPI_Switzerland,PIB_Switzerland,LT_IR_Switzerland,ST_IR_Switzerland,WH_Switzerland,CPI_United_Kingdom,PIB_United_Kingdom,LT_IR_United_Kingdom,ST_IR_United_Kingdom,WH_United_Kingdom
TIME_PERIOD,,,,,,,,,,,,,,,,,,,,,
1995-Q1,NaN,43186.6,7.627367,5.043334,40.00,NaN,53331.5,8.293333,5.706666,37.8,...,NaN,64546.7,5.275333,NaN,NaN,NaN,257589.8,8.661467,6.722410,36.90
1995-Q2,NaN,46786.8,7.184033,4.680000,39.95,NaN,56119.3,7.590000,5.050000,37.8,...,NaN,67655.1,4.947333,NaN,NaN,NaN,248722.5,8.242567,6.742444,36.85
1995-Q3,NaN,46335.5,6.997967,4.373333,39.90,NaN,53741.7,7.186666,4.416667,37.8,...,NaN,67566.8,4.714667,NaN,NaN,NaN,255889.4,8.108933,6.849310,36.80
1995-Q4,NaN,47362.4,6.730933,4.176667,39.85,NaN,57115.3,6.853333,3.960000,37.8,...,NaN,70342.0,4.175667,NaN,NaN,NaN,263308.8,7.788167,6.696187,36.75
1996-Q1,71.553333,44641.2,6.396633,3.476667,39.80,70.003333,54240.0,6.640000,3.350000,37.8,...,NaN,67670.1,4.400333,NaN,37.0,68.1,262343.7,7.753200,6.277534,36.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-Q4,132.593333,122280.3,3.125533,3.957456,32.90,127.550000,158085.7,3.150000,3.957456,34.1,...,106.443333,213788.9,0.841333,1.718106,36.3,NaN,NaN,4.234600,5.326667,NaN
2024-Q1,133.326667,117304.1,2.835333,3.923615,34.00,129.700000,147812.8,2.890000,3.923615,35.0,...,106.703333,212946.8,0.765667,1.628413,35.7,NaN,NaN,4.029300,5.203333,NaN
2024-Q2,134.366667,120746.2,3.013533,3.808172,32.90,131.086667,154840.7,3.063333,3.808172,33.8,...,107.440000,211841.3,0.722333,1.475667,35.9,NaN,NaN,4.201500,5.166667,NaN


## Estimation de l'impact des chocs

Nous allons maintenant estimer avec une régression de panel l'impact des chocs de politique monétaire identifiés, sur différentes variables (GDP, employment ? ...). La méthode utilisé est celle de Jorda (2005), la méthode de projection locale. Cela consiste à "estimer des projections locales à chaque période au lieu d'extrapoler sur des horizons lointains à partir d'un modèle."
Nous allons estimer une équation de la forme suivante, comme dans Iacoviello & Navarro (2018) :

$$
y_{i,t+h} = \alpha_{i,h} + \beta_h u_t + A_{h,i} Z_{i,t} + \varepsilon_{i,t+h}
$$

où :

- $y_{i,t+h}$ est le PIB du pays $ i $ au temps $t$,
- $\alpha_{i,h} $ est un effet fixe spécifique au pays,
- $u_t$ est le choc monétaire,
- $Z_{i,t}$  représente les variables de contrôle : 4 lags du PIB, des tendances linéaire et quadratique,
- $\varepsilon_{i,t+h}$  est le terme d'erreur.

Pour chaque h il faut estimer un $\beta_h$, ensuite il faut faire les IRF (fonctions de réponse impulsionnelle) pour les différents Y qu'on utilise.


Pour les variables de contrôle, il faut sûrement tester lesquelles sont les meilleures, ne pas forcément reproduire exactement le modèle.




<span style="color:red;"> Le code suivant est fait avec chat gpt entièrement, il faut modifier le nom des variables, je ne sais pa du tout s'il va fonctionner ni si c'est exactement le modèle qu'il faut faire. </span>

Définir la fonction d'estimation

In [ ]:
def local_projection_irf(data, response_var, shock_var, control_vars, max_horizon=12):
    """Estime les réponses impulsionnelles (IRF) par projections locales."""
    
    results = []
    
    for h in range(max_horizon + 1):
        data[f'{response_var}_lead_{h}'] = data.groupby('Country')[response_var].shift(-h)
        
        # suppression des lignes contenant des NA :
        df = data.dropna(subset=[f'{response_var}_lead_{h}', shock_var] + control_vars) 
        
        y = df[f'{response_var}_lead_{h}']
        X = df[[shock_var] + control_vars]
        X = sm.add_constant(X)  # Ajouter l'intercept

        model = sm.OLS(y, X).fit(cov_type="cluster", cov_kwds={"groups": df["Country"]})  # Cluster par pays: corrige l’hétéroscédasticité et l'autocorrélation intra-groupe.
        results.append((h, model.params[shock_var], model.bse[shock_var]))  # Stocker horizon, coefficient, erreur standard

    return pd.DataFrame(results, columns=["Horizon", "Beta", "SE"])


Exécuter l'estimation

In [ ]:
control_vars = ['GDP_lag1', 'GDP_lag2', 'GDP_lag3', 'GDP_lag4', 'time', 'time_squared']  # Exemples de contrôles
irf_results = local_projection_irf(data, response_var="GDP", shock_var="Shock", control_vars=control_vars, max_horizon=12)

Tracer les IRF

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(irf_results["Horizon"], irf_results["Beta"], marker="o", label="IRF")
plt.fill_between(irf_results["Horizon"], 
                 irf_results["Beta"] - 1.96 * irf_results["SE"], 
                 irf_results["Beta"] + 1.96 * irf_results["SE"], 
                 color='gray', alpha=0.3, label="IC 95%")
plt.axhline(0, color='black', linestyle='--')
plt.xlabel("Horizon (trimestres)")
plt.ylabel("Effet du choc monétaire")
plt.title("Réponse impulsionnelle du PIB à un choc monétaire")
plt.legend()
plt.show()
